In [83]:
import requests
import pandas as pd
import json
from datetime import datetime

In [84]:
cookies = 'report_ver=3; _fw_crm_v=b03a9b5b-9cdb-4d81-ab85-c51c8076a2ee; _fbp=fb.1.1722010576000.953081039792325240; _gid=GA1.2.241215532.1727445736; _gat_gtag_UA_66880228_3=1; sapo-admin=CfDJ8DA1016STPdNrAaTdK2HgWcOvQq3cBLWhNWAMA8IKlwPAzqHaDKJIUybgJMAUxEEADpxFQiU-5OmIZ8auRs0G1kvGKe-_j1OTvR2uF-uFrK6XaAxM7knZ0Mg8prpERErLNvaZrRwHZ4ECVk80Bf0nj0vcq7wN9_QTvbiqd9K6kmAIndG36OcN7TQN1Xy9spiqvNTh271rZK0vytCUBzGdQk6Ee8OJK7VwOM3RVIZvehWYBp99LztSrUF4UQ0m4IrFnAUa5tcGPudK42oexnr4VVg6rpmhDzxWEfpPcZ0BhznOvMg3x9yLy-rHPh28JN4OJgvs4M4-oQc8h60daI7HGbhs-Xa2nrRzciVC4WN_3Y8yv7lpAfGI01QhqvKXztmoP5TDKduMsY_zdKb7Q-8vwe3iTuG43ezhz_S3Bwy6EkzH6Rqdetajp3okorpYkrqLWCEEBNpxMsjRFIuUHx8y6km7pse9Yv9dZy9tGtCmcHc3oTT7mhO13iVvbuiEmjUMmn5412Es2cLN6HD8kV6EuUChmgQZt2qHJ9DVVUnnCFkJAIscj8--yiEgzArLM9Tvkxhe8zh8by46uxkQOa858ZvMGwog8SdzZuqdgP0fUP5MLdcNJDrvCPLogVHAUcB7g; lang=vi; _admin_session_id=cd1b50f2-7bb3-63c1-7603-a2cb9f3689ee; .AspNetCore.Session=CfDJ8DA1016STPdNrAaTdK2HgWcAQKOHjaPuMKuhR8GIl0wrnb%2B5%2BFQJnhUHcA0CkYqSlI%2Bx18o7HdHSWRrgfjXundNAfqnNaMc%2ByNgrfDnZPyXzUlNethq5uIgQKNi7dl6%2By4YzOKmFZjBZX4BBCbjRYe28kfqYxslMSf6LsIeLKhlW; _ga_4NX0F91DEX=GS1.1.1727445735.53.1.1727445737.0.0.0; _ga=GA1.1.1357561029.1697356741; _ga_Z1V9QDBF2Q=GS1.1.1727445735.53.1.1727445737.0.0.0'

In [85]:
url = "https://beanyfood.mysapogo.com/admin/stock_transfers.json?page=1&limit=20"
payload = {}
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'vi',
  'Connection': 'keep-alive',
  'Cookie': cookies,
  'Referer': 'https://beanyfood.mysapogo.com/admin/stock_transfers.json?page=1&limit=20',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
  'X-Sapo-Client': 'sapo-frontend-v3',
  'X-Sapo-ServiceId': 'sapo-frontend-v3',
  'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"'
}

response = requests.request("GET", url, headers=headers, data=payload)

In [86]:
def get_product_name_sku_unit(variant_id: int) -> (str, str, str):
    product_url = f"https://beanyfood.mysapogo.com/admin/variants/search.json?ids={variant_id}&status=active%2Cinactive%2Cdeleted&page=1&limit=200"
    product_payload = {}
    product_headers = {
      'Accept': 'application/json, text/plain, */*',
      'Accept-Language': 'vi',
      'Connection': 'keep-alive',
      'Cookie': cookies,
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
      'X-Sapo-Client': 'sapo-frontend-v3',
      'X-Sapo-ServiceId': 'sapo-frontend-v3',
      'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"'
    }
    
    product_response = requests.request("GET", product_url, headers=product_headers, data=product_payload)
    product_data = json.loads(product_response.text)
    product_name = product_data["variants"][0]["name"]
    product_sku = product_data["variants"][0]["sku"]
    product_unit = product_data["variants"][0]["unit"]
    return product_name, product_sku, product_unit
    

In [87]:
data: dict = json.loads(response.text)

In [88]:
created_ons = []
product_names = []
product_skus = []
source_locations = []
destination_locations = []
manufacture_dates = []
expiration_dates = []
type_sts = []
quantities = []
product_units = []
codes = []
notes = []

In [89]:
locations = {
    598184: "CN HCM",
    598800: "Xưởng",
    719281: "KIOS",
    727076: "CN Bình Thuận"
}

In [90]:
for stock_transfers in data['stock_transfers']:
    created_on = stock_transfers['created_on']
    created_on = datetime.strptime(created_on,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
    code = stock_transfers['code']
    source_location = locations[stock_transfers["source_location_id"]]
    destination_location = locations[stock_transfers["destination_location_id"]]
    if stock_transfers["destination_location_id"]==727076:
        continue
    for line_item in stock_transfers['line_items']:
        lots_dates = line_item["lots_dates"]
        variant_id = line_item["variant_id"]
        product_name, sku, unit = get_product_name_sku_unit(variant_id)
        for lots_date in lots_dates:
            lots_number = lots_date["lots_number"]
            manufacture_date = lots_date["manufacture_date"]
            manufacture_date = datetime.strptime(manufacture_date,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
            expiration_date = lots_date["expiration_date"]
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
            type_st = "CHUYỂN"
            quantity = lots_date["quantity"]
            #save data
            created_ons.append(created_on)
            product_names.append(product_name)
            product_skus.append(sku)
            source_locations.append(source_location) 
            destination_locations.append(destination_location)
            manufacture_dates.append(manufacture_date)
            expiration_dates.append(expiration_date)
            type_sts.append(type_st) 
            quantities.append(quantity) 
            product_units.append(unit) 
            codes.append(code)
            notes.append("")

In [91]:
data = {
    'Created On': created_ons,
    'Product Name': product_names,
    'Product SKU': product_skus,
    'Source Location': source_locations,
    'Destination Location': destination_locations,
    'Manufacture Date': manufacture_dates,
    'Expiration Date': expiration_dates,
    'Type': type_sts,
    'Quantity': quantities,
    'Product Unit': product_units,
    'Code': codes,
    'Notes': notes
}
df = pd.DataFrame(data)

In [92]:
df.to_csv('stock.csv', index=False)